In [1]:
#https://medium.com/datadriveninvestor/choosing-the-best-algorithm-for-your-classification-model-7c632c78f38f
#https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html
#https://www.dataquest.io/blog/sci-kit-learn-tutorial/
#https://datascience.stackexchange.com/questions/33256/how-to-apply-machine-learning-model-to-new-dataset

In [2]:
from fastai.tabular import *
import os
import pandas as pd 
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import ComplementNB
from sklearn import svm, tree
import xgboost
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier

from sklearn import model_selection
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt

import pickle


In [3]:
path = Path('/Users/bbabu/fastai/DataSamples')

data = pd.read_csv(path/'Output-5v5.csv', sep=',', header=0)
df_test = pd.read_csv(path/'Output-5v5-Test.csv', sep=',', header=0)
df_test_2 = pd.read_csv(path/'Output-5v5-Test.csv', sep=',', header=0)


In [4]:
data = pd.read_csv(path/'Groups_with_Owners-V1.csv', sep=';', header=0)
df_test = pd.read_csv(path/'Groups_without_Owners-V1.csv', sep=';', header=0)
df_test_2 = pd.read_csv(path/'Groups_without_Owners-V1.csv', sep=';', header=0)

data.drop('Description', axis=1, inplace=True)
df_test.drop('Description', axis=1, inplace=True)
df_test_2.drop('Description', axis=1, inplace=True)

In [5]:
data.drop(['Description', 'MemberDN', 'State'], axis=1, inplace=True)
df_test.drop(['Description', 'MemberDN', 'State', 'Owner'], axis=1, inplace=True)
df_test_2.drop(['Description', 'MemberDN', 'State', 'Owner'], axis=1, inplace=True)

In [6]:
#data.dtypes
#data['Title'].isnull().sum()

In [7]:
le = preprocessing.LabelEncoder()
categorical = list(data.select_dtypes(include=['object']).columns.values)
for cat in categorical:
    #print(cat)
    data[cat].fillna(0, inplace=True)
    data[cat] = le.fit_transform(data[cat].astype(str))


In [8]:
le = preprocessing.LabelEncoder()

categorical = list(df_test_2.select_dtypes(include=['object']).columns.values)
for cat in categorical:
    #print(cat)
    df_test_2[cat].fillna(0, inplace=True)
    df_test_2[cat] = le.fit_transform(df_test_2[cat].astype(str))

In [9]:
#Create Dependent and Independent Datasets based on our Dependent #and Independent features
X = data[['GroupDN', 'Member', 'Manager', 'BusinessGroup', 'Platform', 'Title', 'Department', 'Country', 'City']]
y = data['Owner']
#Split the Data into Training and Testing sets with test size as #50%
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.80, shuffle=False)

In [10]:
model = GaussianNB(priors=[0.99, 0.01], var_smoothing=1e-05) #le-01 is final, this is better
# model = GaussianNB(var_smoothing=1e-01) #le-01 is final, this is better
model.fit(X_train, y_train)

GaussianNB(priors=[0.99, 0.01], var_smoothing=1e-05)

In [11]:
# Save model to disk
filename = path/'Final_Model-GNB.sav'
pickle.dump(model, open(filename, 'wb'))

In [12]:
# filename = path/'Final_Model-GNB-Saved.sav'

In [13]:
# Load model from disk and use it to make new predictions
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
# print(result)

In [14]:
X_train = df_test_2
pred = loaded_model.predict(X_train)
# print(pred)
# pred.sum()

In [15]:
final_df = pd.DataFrame({'GroupDN': df_test['GroupDN'],'Member': df_test['Member'], 'Manager': df_test['Manager'], 'Owner': pred})
final_df.to_csv(path/'MLProcessed1-Sklearn-GNB.csv', header=True, index=False)

In [16]:
grouped = final_df.groupby(['GroupDN'])

In [17]:
i = 0
for name, group in grouped:
    if group['Owner'].sum() == 0:
        i += 1
        

In [18]:
# print("Feature importance table is as follows:")
# print(feature_imp.to_string())
# print(pred)
print("Model accuracy is: %s"%(result))
print("Total number of predicted owners are: %s"%(pred.sum()))
print("Mean of owners are: %s"%(pred.mean()))
print("Total number of groups are: %s"%(len(grouped)))
print("Number of predicted groups with owners are: %s"%(len(grouped) - i))
# print("Accuracy score of the model is: %s"%(acc))

Model accuracy is: 0.9788353476511502
Total number of predicted owners are: 0
Mean of owners are: 0.0
Total number of groups are: 48963
Number of predicted groups with owners are: 0
